In [ ]:
import pandas as pd
import os
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import behaviour_functions as bhv
import statsmodels.api as sm
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from pylab import *
import math
import statistics
import itertools
from scipy import stats
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
from statsmodels.formula.api import ols
from pandas.api.types import CategoricalDtype
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import behaviour_functions as bhv

In [ ]:
path = '/Volumes/lab-kohlj/home/shared/people/Irene/sensory_deprivation/mesh/BORIS project/scored/'
IDs = ["BRYB120.5c", "BRYB138.1d", "BRYB138.1e", "BRYC109.5f", "BRYC115.2e", "BRYD69.5g", "BRYD69.5h", "BRYD75.3g"]
times = ["baseline", "day4"]

In [ ]:
# Load data
df = pd.read_csv('/Users/salgari/Documents/Documents - XJK222YYCT/GitHub/sensitization_behaviour/dataframes/mesh_behaviours_df.csv')

# Prepare dataset for analysis
df = df.drop(columns=['Unnamed: 0', 'duration']) # drop unnecessary columns
df['event'] = np.where(df['latency'] < 900, '1', '0') # create event column
df['time'] = df['time'].replace('baseline', 'day0') # use the replace method to change 'baseline' to 'day0'

try: 
    # extract the numeric part of the time values and convert them to integers
    df['time'] = df['time'].str.extract('(\d+)', expand=False).astype(int)
    
except:
    print('sometyhing went wrong, check that baseline has been replaced with day0')

# separate the dataframe into different timepoint groups
day_dfs = {}
for i in df.time.unique():
    day_dfs['day' + str(i)] = df[(df.time == i) & (df.behaviour == 'retrieval to nest')]


In [ ]:
# Run Kaplan-Meier analysis on each DataFrame in day_dfs
kmfs = {}
for key, df in day_dfs.items():
    kmf = KaplanMeierFitter()
    kmf.fit(durations = df['latency'], event_observed = df['event'], label=key)
    kmfs[key] = kmf

In [ ]:
# Perform log-rank test on two selected DataFrames

for key1, df1 in day_dfs.items():
    for key2, df2 in day_dfs.items():
        if key1 != key2:
            result = logrank_test(df1['latency'], df2['latency'], 
                                  event_observed_A = df1['event'], event_observed_B=df2['event'])
            print(result.print_summary())
            print(result.p_value)
            print(result.test_statistic)

            # Plot the survival curves
            kmfs['day0'].plot_cumulative_density(ci_show=False,label=key1,
                                                linewidth=3,color='#d7301f',legend=False, alpha=.7)
            kmfs['day4'].plot_cumulative_density(ci_show=False,label=key2,
                                                linewidth=3,color='grey',legend=False, alpha=.7)

            plt.ylim(-0.1,1.1)
            params = {'mathtext.default': 'regular' }          
            plt.rcParams.update(params)
            plt.ylabel('$P_{retrieval}$', multialignment='center',labelpad=5)
            plt.xlabel('Time (s)', labelpad=5)
            sns.despine()
            plt.locator_params(axis='x', nbins=4)
            plt.ylim(0,1)
